In [49]:
import pandas as pd
import nltk as nltk
import texthero as hero
from sklearn.model_selection import train_test_split

import re
from nltk.tokenize import word_tokenize
import gensim
import string
from nltk.probability import FreqDist
from tensorflow.keras.optimizers import Adam


In [50]:
df = pd.read_csv('text_emotion.csv')

dfTweet = df.drop(['tweet_id','author'],axis = 1)
dfTweet=dfTweet.applymap(str.lower)

dfJournal = pd.read_csv('Emotion_final.csv')
dfJournal=dfJournal.applymap(str.lower)

dfgo = pd.read_csv('go_emotions_dataset.csv')
dfTweet.rename(columns={'sentiment': 'Emotion', 
                           'content': 'Text'}, inplace=True)

In [51]:
dfgo = dfgo.drop(dfgo[dfgo.example_very_unclear == True].index)
RangeEmotiondfgo = ['admiration','amusement','anger','annoyance','approval','caring','confusion','curiosity','desire','disappointment','disapproval','disgust','embarrassment','excitement','fear','gratitude','grief','joy','love','nervousness','optimism','pride','realization','relief','remorse','sadness','surprise','neutral']

In [52]:
dfgo['Emotion'] = dfgo[RangeEmotiondfgo].idxmax(axis=1)
dfgo = dfgo.drop(RangeEmotiondfgo,axis=1)
dfgo = dfgo.drop(['example_very_unclear','id'],axis=1)
dfgo

,Text,Emotion
0,That game hurt.,sadness
2,"You do right, if you don't care then fuck 'em!",neutral
3,Man I love reddit.,love
4,"[NAME] was nowhere near them, he was by the Fa...",neutral
5,Right? Considering it’s such an important docu...,gratitude
...,...,...
211219,"Well, I'm glad you're out of all that now. How...",joy
211220,Everyone likes [NAME].,love
211221,Well when you’ve imported about a gazillion of...,caring
211222,That looks amazing,admiration


In [53]:
dfgo.Emotion.value_counts()
dfgo

,Text,Emotion
0,That game hurt.,sadness
2,"You do right, if you don't care then fuck 'em!",neutral
3,Man I love reddit.,love
4,"[NAME] was nowhere near them, he was by the Fa...",neutral
5,Right? Considering it’s such an important docu...,gratitude
...,...,...
211219,"Well, I'm glad you're out of all that now. How...",joy
211220,Everyone likes [NAME].,love
211221,Well when you’ve imported about a gazillion of...,caring
211222,That looks amazing,admiration


In [54]:
# dfJournal.Emotion.value_counts()
# dfJournal

In [55]:
dfJournal = dfJournal.append(dfTweet)
dfTweet = 0
dfJournal = dfJournal.append(dfgo)
dfgo = 0

In [56]:
dfJournal.Emotion.replace({'happy': 'happiness'}, regex=True,inplace=True)
#dfJournal.Emotion.replace({'fear': 'anger'}, regex=True,inplace=True)
#dfJournal.Emotion.replace({'joy': 'happiness'}, regex=True,inplace=True)
#dfJournal.Emotion.replace({'fun': 'happiness'}, regex=True,inplace=True)
#dfJournal.Emotion.replace({'disgust': 'anger'}, regex=True,inplace=True)
dfJournal = dfJournal.reset_index()
# dfJournal.Emotion.replace({'empty': 'neutral'}, regex=True,inplace=True)
# dfJournal.Emotion.replace({'boredom': 'neutral'}, regex=True,inplace=True)
# dfJournal.Emotion.replace({'hate': 'sadness'}, regex=True,inplace=True)
# dfJournal.Emotion.replace({'worry': 'surprise'}, regex=True,inplace=True)


df = dfJournal[dfJournal.Emotion == 'happiness'].index
df = df.append(dfJournal[dfJournal.Emotion == 'anger'].index)
df = df.append(dfJournal[dfJournal.Emotion == 'love'].index)
df = df.append(dfJournal[dfJournal.Emotion == 'sadness'].index)
# dfJournal

df2 = dfJournal.iloc[df]


In [57]:
#df = pd.DataFrame(df)
df = df2.reset_index()

In [58]:
df = df.drop(['level_0','index'],axis = 1)
df.Emotion.value_counts()

sadness      15293
happiness    12238
anger        11059
love         10793
Name: Emotion, dtype: int64

In [59]:
categoriesLabel = df['Emotion'].unique()
df.Emotion.replace({'happiness': 0}, regex=True,inplace=True)
df.Emotion.replace({'anger': 1}, regex=True,inplace=True)
df.Emotion.replace({'love': 2}, regex=True,inplace=True)
df.Emotion.replace({'sadness': 3}, regex=True,inplace=True)

nb_labels = len(dfJournal.Emotion.value_counts())


In [60]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop = list(set(stop) - set(['not','no']))
lemmatizer = nltk.stem.WordNetLemmatizer()

def tweets_cleaning(tweet, remove_emojis=True):
    """Apply function to a clean a tweet"""
    tweet = tweet.lower().strip()
    # romove urls
    url = re.compile(r'https?://\S+|www\.\S+')
    tweet = url.sub(r'',tweet)
    # remove html tags
    html = re.compile(r'<.*?>')
    tweet = html.sub(r'',tweet)
    # remove punctuation
    operator = str.maketrans('','',string.punctuation) #????
    tweet = tweet.translate(operator)
    if remove_emojis:
        tweet = tweet.encode('ascii', 'ignore').decode('utf8').strip()
    lambda tweet: re.sub(r'https?:\/\/\S+', '', tweet)
    lambda tweet: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', tweet)
    tweet=re.sub(r'@[A-Za-z0-9]+'," ",tweet) ##Removing the usernames
    tweet=re.sub(r'^[A-Za-z0-9.!?]+'," ",tweet) ##Removing digits and punctuations
    tweet=re.sub(r'https?://[A-Za-z0-9./]+'," ",tweet) ## removing links
    tweet=re.sub(r' +'," ",tweet)
    tweet = tweet.lower()
    tweet = re.sub(r"\'s", " ", tweet)
    tweet = re.sub(r"\'ve", " have ", tweet)
    tweet = re.sub(r"can't", "cannot ", tweet)
    tweet = re.sub(r"n't", " not ", tweet)
    tweet = re.sub(r"\'d", " would ", tweet)
    tweet = re.sub(r"\'ll", " will ", tweet)
    tweet = re.sub(r"\'scuse", " excuse ", tweet)
    tweet = tweet.strip(' ')
    tweet = tweet.strip('. .')
    tweet = tweet.replace('.',' ')
    tweet = tweet.replace('-',' ')
    tweet = tweet.replace("’", "'").replace("′", "'").replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")
    tweet = tweet.replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")
    tweet = tweet.replace("€", " euro ").replace("'ll", " will")
    tweet = tweet.replace("don't", "do not").replace("didn't", "did not").replace("im","i am").replace("it's", "it is")
    tweet = tweet.replace(",000,000", "m").replace("n't", " not").replace("what's", "what is")
    tweet = tweet.replace(",000", "k").replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")
    tweet = tweet.replace("he's", "he is").replace("she's", "she is").replace("'s", " own")
    tweet = tweet.replace("didnt","did not")
    tweet = re.sub('\s+', ' ', tweet)
    tweet=tweet.split()
    tweet=[lemmatizer.lemmatize(word) for word in tweet if word not in stop]
    tweet=' '.join(word for word in tweet)
    return tweet.lower()

In [61]:

df['clean_text'] = df['Text'].apply(tweets_cleaning)

train_data=df.iloc[:,-1].values
df['clean_text']=train_data
df

,Text,Emotion,clean_text
0,i have been with petronas for years i feel tha...,0,petronas year feel petronas performed well mad...
1,i do feel that running is a divine experience ...,0,feel running divine experience expect type spi...
2,i have immense sympathy with the general point...,0,ammense sympathy general point possible proto ...
3,i do not feel reassured anxiety is on each side,0,not feel reassured anxiety side
4,i have the feeling she was amused and delighted,0,feeling amused delighted
...,...,...,...
49378,I pity them more than anything. My life's actu...,3,pity anything life actually pretty okay genera...
49379,"Everytime I see this, I can't help but feel so...",3,see cant help feel someone force cat paw brownie
49380,"Sadly no, everything was thrown around the roo...",3,no everything thrown around room everyti ame r...
49381,"LUL, this is the 3rd time that Ajax has lost t...",3,3rd ti ame ajax lost team busan unlucky


In [62]:
import numpy as np


In [63]:
idx=np.random.randint(0,len(df),1200)
Y_array = np.array(df["Emotion"])
X_array = np.array(df['clean_text'])


X_test=X_array[idx[0:1000]]
y_test=Y_array[idx[0:1000]]
X_virgin = X_array[idx[1001:1200]]
y_virgin = Y_array[idx[1001:1200]]
X_train=np.delete(X_array,idx,axis=0)
y_train=np.delete(Y_array,idx,axis=0)

In [64]:
y_test

array([1, 2, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 3, 0, 1, 0, 1, 0, 2, 1, 2, 3,
       0, 2, 3, 0, 2, 1, 1, 2, 0, 1, 3, 3, 3, 3, 1, 2, 0, 1, 2, 0, 0, 3,
       0, 1, 1, 2, 0, 1, 3, 1, 3, 3, 0, 0, 3, 1, 2, 3, 3, 2, 3, 0, 3, 2,
       3, 0, 2, 2, 1, 3, 3, 2, 0, 3, 3, 0, 1, 2, 1, 1, 0, 3, 3, 0, 3, 0,
       1, 0, 3, 3, 1, 1, 1, 2, 1, 0, 2, 1, 2, 0, 1, 3, 1, 3, 2, 1, 0, 0,
       1, 2, 3, 3, 0, 0, 3, 1, 3, 3, 3, 2, 1, 2, 3, 3, 2, 1, 3, 2, 3, 2,
       3, 2, 1, 0, 3, 2, 2, 0, 2, 3, 0, 3, 2, 1, 3, 1, 1, 2, 1, 3, 2, 1,
       3, 1, 0, 1, 0, 3, 0, 2, 2, 2, 2, 3, 1, 1, 1, 2, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 3, 2, 1, 3, 1, 3, 3, 2, 3, 2, 0, 2, 3, 1, 1,
       1, 2, 1, 2, 0, 1, 1, 1, 3, 1, 3, 3, 0, 2, 2, 1, 2, 2, 1, 3, 1, 3,
       2, 1, 3, 2, 3, 3, 2, 0, 1, 3, 0, 0, 3, 2, 2, 1, 2, 2, 3, 0, 1, 3,
       3, 0, 1, 3, 3, 0, 3, 2, 3, 0, 2, 3, 2, 3, 0, 1, 1, 1, 3, 0, 2, 3,
       0, 0, 0, 0, 0, 2, 0, 2, 1, 3, 3, 3, 3, 1, 3, 1, 0, 0, 3, 0, 0, 3,
       3, 0, 0, 3, 1, 3, 1, 1, 2, 3, 2, 3, 0, 1, 3,

# Vectorizer

In [65]:
from tensorflow.keras.layers import TextVectorization
import tensorflow as tf
vectorizer = TextVectorization(max_tokens=40000,output_mode='int', output_sequence_length=35,standardize='lower_and_strip_punctuation')





In [66]:
vectorizer.adapt(X_train)
X_train

array(['feel running divine experience expect type spiritual encounter',
       'ammense sympathy general point possible proto writer trying find ti ame write corner life no sign agent let alone publishing contract feel little precious',
       'feeling amused delighted', ...,
       'no everything thrown around room everyti ame rebuilt something would annihilated',
       '3rd ti ame ajax lost team busan unlucky',
       'got banned participating brigade'], dtype=object)

In [67]:
len(vectorizer.get_vocabulary())

29861

In [68]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [69]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip

In [70]:
path_to_glove_file = "glove.twitter.27B.100d.txt"
import numpy as np

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 1193514 word vectors.


In [71]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 22471 words (7390 misses)


In [72]:
X_train

array(['feel running divine experience expect type spiritual encounter',
       'ammense sympathy general point possible proto writer trying find ti ame write corner life no sign agent let alone publishing contract feel little precious',
       'feeling amused delighted', ...,
       'no everything thrown around room everyti ame rebuilt something would annihilated',
       '3rd ti ame ajax lost team busan unlucky',
       'got banned participating brigade'], dtype=object)

# Model

In [73]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    #embeddings_initializer = "uniform",
    trainable=False,
)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, restore_best_weights=True)

#int_sequences_input = tf.keras.Input(shape=(None,), dtype="int64")

input = tf.keras.Input(shape=(1,), dtype='string')

x = vectorizer(input)
#x = layers.Dropout(0.2)(x)
x = embedding_layer(x)
x = layers.Dropout(0.1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.2)(x)
preds = layers.Dense(nb_labels, activation="softmax")(x)
model = tf.keras.Model(input, preds)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 35)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 35, 100)           2986300   
                                                                 
 dropout_4 (Dropout)         (None, 35, 100)           0         
                                                                 
 conv1d_3 (Conv1D)           (None, 35, 128)           12928     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 7, 128)           0         
 1D)                                                       

In [83]:
input = tf.keras.Input(shape=(1,), dtype='string')

x = vectorizer(input)
#x = layers.Dropout(0.2)(x)
x = embedding_layer(x)
x = layers.Dropout(0.1)(x)
x = layers.Bidirectional(tf.keras.layers.LSTM(100,return_sequences=True))(x)
x = layers.Bidirectional(tf.keras.layers.LSTM(100))(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)
preds = layers.Dense(nb_labels, activation="softmax")(x)
modelLSTM = tf.keras.Model(input, preds)
modelLSTM.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 35)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 35, 100)           2986300   
                                                                 
 dropout_8 (Dropout)         (None, 35, 100)           0         
                                                                 
 bidirectional_4 (Bidirectio  (None, 35, 200)          160800    
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 200)              2408

In [84]:
# x_train = vectorizer(np.array([[s] for s in X_train])).numpy()
# x_val = vectorizer(np.array([[s] for s in X_test])).numpy()
# x_virgin = vectorizer(np.array([[s] for s in X_virgin])).numpy()

y_train = np.array(y_train)
y_val = np.array(y_test)
y_virgin = np.array(y_virgin)

In [85]:
modelLSTM.compile(
    loss="sparse_categorical_crossentropy", optimizer=Adam(learning_rate=0.0001), metrics=["acc"]
)
modelLSTM.fit(X_train, y_train, batch_size=128, epochs=100, validation_data=(X_test, y_val),callbacks=[callback])

Epoch 1/100
377/377 [==============================] - 81s 195ms/step - loss: 1.7145 - acc: 0.3743 - val_loss: 1.0412 - val_acc: 0.5750
Epoch 2/100
377/377 [==============================] - 66s 175ms/step - loss: 1.0743 - acc: 0.5587 - val_loss: 0.9598 - val_acc: 0.6100
Epoch 3/100
377/377 [==============================] - 63s 167ms/step - loss: 0.9759 - acc: 0.6063 - val_loss: 0.9010 - val_acc: 0.6350
Epoch 4/100
377/377 [==============================] - 64s 169ms/step - loss: 0.9263 - acc: 0.6251 - val_loss: 0.8530 - val_acc: 0.6520
Epoch 5/100
377/377 [==============================] - 68s 179ms/step - loss: 0.8908 - acc: 0.6424 - val_loss: 0.8400 - val_acc: 0.6630
Epoch 6/100
377/377 [==============================] - 75s 200ms/step - loss: 0.8620 - acc: 0.6532 - val_loss: 0.8137 - val_acc: 0.6800
Epoch 7/100
377/377 [==============================] - 79s 208ms/step - loss: 0.8353 - acc: 0.6642 - val_loss: 0.7983 - val_acc: 0.6770
Epoch 8/100
377/377 [===========================

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer=Adam(learning_rate=0.00005), metrics=["acc"]
)
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data=(X_test, y_val),callbacks=[callback])

Epoch 1/100
377/377 [==============================] - 6s 15ms/step - loss: 1.9323 - acc: 0.2894 - val_loss: 1.2693 - val_acc: 0.4900
Epoch 2/100
377/377 [==============================] - 6s 15ms/step - loss: 1.2142 - acc: 0.4642 - val_loss: 1.0321 - val_acc: 0.5910
Epoch 3/100
377/377 [==============================] - 6s 15ms/step - loss: 1.0408 - acc: 0.5625 - val_loss: 0.9399 - val_acc: 0.6350
Epoch 4/100
377/377 [==============================] - 5s 14ms/step - loss: 0.9628 - acc: 0.6031 - val_loss: 0.8890 - val_acc: 0.6420
Epoch 5/100
377/377 [==============================] - 5s 14ms/step - loss: 0.9184 - acc: 0.6263 - val_loss: 0.8542 - val_acc: 0.6640
Epoch 6/100
377/377 [==============================] - 5s 14ms/step - loss: 0.8851 - acc: 0.6415 - val_loss: 0.8300 - val_acc: 0.6740
Epoch 7/100
377/377 [==============================] - 5s 14ms/step - loss: 0.8609 - acc: 0.6539 - val_loss: 0.8070 - val_acc: 0.6820
Epoch 8/100
377/377 [==============================] - 5s 14ms

In [ ]:
predicted_Y = modelLSTM.predict(X_virgin)
y_classes = predicted_Y.argmax(axis=-1)
y_classes
from sklearn import metrics
categories = ['happy','sadness','fear','love']
print(metrics.classification_report(y_virgin, y_classes, target_names=categories))

              precision    recall  f1-score   support

       happy       0.69      0.74      0.71        50
     sadness       0.58      0.69      0.63        36
        fear       0.76      0.58      0.66        55
        love       0.70      0.72      0.71        58

    accuracy                           0.68       199
   macro avg       0.68      0.69      0.68       199
weighted avg       0.69      0.68      0.68       199



In [ ]:
#modelLSTM.save('model_KaggleLSTM')

In [ ]:
# from keras import models
# model_2 = models.load_model('model_KaggleLSTM')

In [ ]:
predicted_Y = model.predict(X_virgin)
y_classes = predicted_Y.argmax(axis=-1)
y_classes
from sklearn import metrics
categories = ['happy','sadness','fear','love']
print(metrics.classification_report(y_virgin, y_classes, target_names=categories))

              precision    recall  f1-score   support

       happy       0.72      0.82      0.77        50
     sadness       0.63      0.81      0.71        36
        fear       0.81      0.64      0.71        55
        love       0.79      0.72      0.76        58

    accuracy                           0.74       199
   macro avg       0.74      0.75      0.74       199
weighted avg       0.75      0.74      0.74       199



In [ ]:

test = ['I feel a bit sad today','Today, mom died','Jane and I argued yesterday, despite that ','I am not happy']
test = tweets_cleaning(test)
predicted_Y = modelLSTM.predict(test)
y_classes = predicted_Y.argmax(axis=-1)
y_classes

ValueError: Number of classes, 4, does not match size of target_names, 5. Try specifying the labels parameter